This notebook loads the files as `AudioSegment`, trims silences, and writes them as mono-channel .wav at 16 kHz.

In [1]:
from os import walk
from pydub import AudioSegment, effects
from pydub.silence import detect_leading_silence
from tqdm import tqdm

In [2]:
# %load_ext ipycache
# %load_ext blackcellmagic

In [3]:
DISCOURSE_CONTEXT = {
    "aesdd": False,
    "BAUM1": False,
    "BAUM2": True,
    "cafe": False,
    "CREMA-D": False,
    "dzafic": False,
    "ekorpus": False,
    "EmoDB": False,
    "EmoReact_V_1.0": True,
    "Emotional_EMA": False,
    "EmoV-DB_sorted": False,
    "enterface_db": False,
    "esd": False,
    "EYASE": True,
    "jl-corpus": False,
    "LimaCastroScott": False,
    "LEGOv2": True,
    "MAV": False,
    "MELD": True,
    "MESS": False,
    "oreau2": False,
    "ravdess": False,
    "savee": False,
    "ShEMO": True,
    "tess": False,
    "urdu": True,
    "vivae": False,
}

In [4]:
IN_FOLDER = "data/interim"
OUT_FOLDER = "data/preprocessed"

In [5]:
list_of_files = [file_ for _, _, files in walk(IN_FOLDER) for file_ in files]
len(list_of_files)

87364

In [6]:
datasets = set(DISCOURSE_CONTEXT.keys())
samples = []
# Take 1 sample from each dataset
for file in list_of_files:
    if (dataset_name := file.split("+", maxsplit=2)[1]) in datasets:
        datasets.remove(dataset_name)
        samples.append(file.strip())
    if not datasets:
        break
samples

['00000+aesdd+aesdd.1+f+ang+-1+ell+el-gr.wav',
 '00604+BAUM1+BAUM1.s001+m+ang+-1+tur+tr-tr.mp4',
 '01908+BAUM2+BAUM2.S002+m+con+-1+eng+en.wav',
 '02255+cafe+cafe.01+m+ang+-1+fra+fr-ca.aiff',
 '03191+CREMA-D+CREMA-D.1005+m+ang+-1+eng+en-us.wav',
 '10064+dzafic+dzafic.0+f+ang+-1+eng+en-au.avi',
 '10070+ekorpus+ekorpus.0+f+hap+1+est+et-ee.wav',
 '10937+EmoDB+EmoDB.03+m+hap+1+deu+de-de.wav',
 '11472+EmoReact_V_1.0+EmoReact_V_1.0.train4+m+unc+-1+eng+en.mp4',
 '12409+Emotional_EMA+Emotional_EMA.abe+m+ang+-1+eng+en-us.wav',
 '13031+EmoV-DB_sorted+EmoV-DB_sorted.bea+f+amu+1+eng+en-us.wav',
 '18203+enterface_db+enterface_db.1+m+ang+-1+eng+en.avi',
 '19496+esd+esd.0001+f+sur+-1+cmn+zh-cn.wav',
 '54496+EYASE+EYASE.Male03+m+ang+-1+arz+ar-eg.wav',
 '55075+jl-corpus+jl-corpus.female1+f+ang+-1+eng+en-nz.wav',
 '56755+LEGOv2+LEGOv2.20061123)000+f+neu+0+eng+en-us.wav',
 '60998+LimaCastroScott+LimaCastroScott.C+m+ach+1+por+pt-pt.wav',
 '61115+MAV+MAV.42+m+ang+-1+fra+fr-ca.wav',
 '61205+MELD+MELD.Chloe+f

In [7]:
def load_file(in_path: str) -> AudioSegment:
    "Load and return an AudioSegment from file."
    try:
        return AudioSegment.from_file(
            in_path, format=in_path.rsplit(".", maxsplit=1)[-1]
        )
    except:
        try:
            return AudioSegment.from_file(in_path)
        except:
            try:
                return AudioSegment.from_file(in_path, format="wav")
            except:
                try:
                    return AudioSegment.from_wav(in_path)
                except:
                    try:
                        return AudioSegment.from_file_using_temporary_files(in_path)
                    except:
                        try:
                            return AudioSegment.from_file(in_path, format="aac")
                        except:
                            return AudioSegment.from_file(in_path, codec="pcm_u16be")

In [8]:
trim_leading_silence: AudioSegment = lambda x: x[
    detect_leading_silence(x, silence_threshold=-60.0) :
]
# detect_leading_silence(_) should return 0 if no leading silence found.
# detect_leading_silence() should return len(_) if no end is found (it's all silent) and raise IndexError as a result
# default silence_threshold is -50.0 dFBS; lowering it reduces the amount of information lost (in theory)

In [9]:
def process_audio(file_: str) -> int:
    "Load, trim, set sample width, set to mono, set frame rate, and write a sound file to OUT_FOLDER."
    "Returns 1 if zero duration detected after trim operations, 2 if longer than 1 minute, or 0 after export."

    # Load
    _ = load_file(in_file := f"{IN_FOLDER}/{file_}")

    # Sample width 16 bits
    try:
        _ = _.set_sample_width(2)
    except:
        _ = AudioSegment.from_file_using_temporary_files(in_file).set_sample_width(2)
    # Normalize
    _ = effects.normalize(_)

    # Trim silence
    try:
        _ = trim_leading_silence(_)
        if not DISCOURSE_CONTEXT[file_.split("+", maxsplit=2)[1]]:
            _ = trim_leading_silence(_.reverse()).reverse()
    except IndexError:
        print("all silence detected during trim for {file_}")
        return 1
    # Filter short and long
    if (duration := len(_)) < 200:
        print(f"{file_} is less than 200 ms after trim")
        return 1
    if duration > 60000:
        print(f"{file_} exceeds 1 minute in duration")
        return 2
    # Set mono 16 kHz
    _ = _.set_channels(1)
    _ = _.set_frame_rate(16000)

    # Write
    _.export(
        out_f=f"{OUT_FOLDER}/{file_.rsplit('.', maxsplit=1)[0]}.wav",
        format="wav",
        codec="pcm_s16le",
        bitrate="128k",
    )

    return 0

In [10]:
for sample_file in samples:
    if process_audio(sample_file):
        break
else:
    print("no problems so far")

no problems so far


In [11]:
bad_files, short_files, long_files = [], [], []

for sample_file in tqdm(list_of_files):
    try:
        if (return_code := process_audio(sample_file)) == 1:
            short_files.append(sample_file)
        elif return_code == 2:
            long_files.append(sample_file)
    except Exception as e:
        bad_files.append((sample_file, e))
else:
    print("no problems in conversion")

  1%|          | 884/87364 [03:56<6:48:17,  3.53it/s]

00883+BAUM1+BAUM1.s001+m+hap+1+tur+tr-tr.mp4 is less than 200 ms after trim


  1%|          | 897/87364 [04:00<6:41:08,  3.59it/s]

00896+BAUM1+BAUM1.s002+m+hap+1+tur+tr-tr.mp4 is less than 200 ms after trim


  1%|          | 931/87364 [04:11<6:34:32,  3.65it/s]

...

74673+MELD+MELD.Ross+m+hap+1+eng+en-us.mp4 is less than 200 ms after trim


 86%|████████▌ | 74783/87364 [4:42:18<1:31:00,  2.30it/s]

74782+MELD+MELD.Joey+m+ang+-1+eng+en-us.mp4 is less than 200 ms after trim


100%|██████████| 87364/87364 [5:08:17<00:00,  4.72it/s]

no problems in conversion


In [12]:
print(f"There were {len(short_files)} files discarded because they were shorter than 200 milliseconds.")

There were 211 files discarded because they were shorter than 200 milliseconds.


In [13]:
print(f"There were {len(long_files)} files discarded because duration exceeded 1 minute.")

There were 7 files discarded because duration exceeded 1 minute.


In [14]:
print(f"There were {len(bad_files)} bad files.")

There were 0 bad files.


In [15]:
if bad_files:
    print(*bad_files, sep="\n")